In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_quad2Diter
from python.simulators import Simulator
# from python.mProfile import mProfile

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [10, 0, 0]

psf_vec = PsfVectorial() 
psf_vec.zerooffset=0.000  # True zero

sim = Simulator(fluorophores=fl)

numberOfLocalizations=1000

In [ ]:
# define scan pattern
L = 75  # size of scan pattern
orbitpoints = 6  # number of probing points in orbit
probecenter = True  # should we also probe the center?
laserpower = 5  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 2 # how often to repeat the pattern scan
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [ ]:
# we need an estimator. Define as component
sim.defineComponent("estdonut", "estimator", est_quad2Diter, parameters=[L, probecenter], dim=(0,1))

In [ ]:
seq = ["donut", "estdonut"]
# mProfile.profileOn(['utilities.py', 'psf.py', 'psf_vectorial.py', 'simulator.py'])

out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
# mProfile.profileOff()
# mProfile.report()
# out.loc: localizations
# out.fluorophores: position of fluorophores
# out.raw: photon measurements

sigmaCRB=sim.calculateCRBpattern("donut", dim=(0,1))/np.sqrt(np.mean(out.loc.phot))

print("vectorial PSF:")
sim.summarize_results(out)

psf0 = psf_vec.imagestack("vortex")


In [ ]:
from python.tools import imx

imx(psf0)

In [ ]:
psf_vec.zerooffset=0.005
out=sim.runSequence(seq)
print(f"zero offset = {psf_vec.zerooffset}:")
sim.summarize_results(out)

In [ ]:
psf_vec2 = PsfVectorial()

In [ ]:
# Add Zernike:
# Zr[k,0]: n, Zr[k,1]: m, Zr[k,2]: amplitude as fraction of wavelength
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((2,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.1   # spherical aberrations
sys_aberr['Zr'][1,0], sys_aberr['Zr'][1,1], sys_aberr['Zr'][1,2] = 2, 2, 0.05  # astigmatism
psf_vec2.setpar(**sys_aberr)
sim.definePattern("donut_aber", psf_vec2, 
                  phasemask="vortex", 
                  makepattern="orbitscan", 
                  orbitpoints=4,
                  probecenter=probecenter,
                  orbitL=L,
                  pointdwelltime=pointdwelltime,
                  laserpower=laserpower,
                  repetitions=repetitions)
seq={"donut_aber","estdonut"}
out=sim.runSequence(seq)